# Assignment 4.1

## Reading S3 bucket into SageMaker Studio

In [14]:
import boto3
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from collections import Counter
from io import StringIO
from sklearn.model_selection import train_test_split

s3_client = boto3.client("s3")

BUCKET='ads508projectbucket'

# For Country_and_Subscriber_df
Country_and_Subscriber_Key='Aggregated_Metrics_By_Country_And_Subscriber_Status.csv'
Country_and_Subscriber_response = s3_client.get_object(Bucket=BUCKET, Key=Country_and_Subscriber_Key)
Country_and_Subscriber_df = pd.read_csv(Country_and_Subscriber_response.get("Body"))

# For Video_df
Video_Key='Aggregated_Metrics_By_Video.csv'
Video_response = s3_client.get_object(Bucket=BUCKET, Key=Video_Key)
Video_df = pd.read_csv(Video_response.get("Body"))

# For Comments_df
Comments_Key='All_Comments_Final.csv'
Comments_response = s3_client.get_object(Bucket=BUCKET, Key=Comments_Key)
Comments_df = pd.read_csv(Comments_response.get("Body"))

# For Performance_df
Performance_Key='Video_Performance_Over_Time.csv'
Performance_response = s3_client.get_object(Bucket=BUCKET, Key=Performance_Key)
Performance_df = pd.read_csv(Performance_response.get("Body"))

## Transforming Data for Training

### Comments_df

For this data set, we want to find the max reply count and the max like count for a comment in each video. To do this, we will group by unique Video ID and find the max. 

In [4]:
Comments_altered = Comments_df[['VidId', 'Reply_Count', 'Like_Count']]
grouped_comments = Comments_altered.groupby(by = 'VidId', as_index = False).max()
transformed_comments = grouped_comments[['VidId', 'Reply_Count', 'Like_Count']]
transformed_comments.rename(columns = {'Reply_Count': 'Reply_Comment_Count', 'Like_Count': 'Like_Comment_Count', 'VidId': 'External Video ID'}, inplace = True)

In [5]:
transformed_comments.head()

,External Video ID,Reply_Comment_Count,Like_Comment_Count
0,-3d1NctSv0c,3,5
1,-ONQ628CXKQ,5,31
2,-kX2b6TF_9k,2,2
3,-pdXWmj9xxU,3,9
4,-zbLpoJVBMI,4,17


This is the only useful information needed from this data set, so we will keep this as is and move onto the next transformation. 

### Country_and_Subscriber_df

We saw from our EDA that the United States and Inda were the two countries which watched the most of Ken Jee's channel. Knowing this, we can look at the percentage of views from the USA and IN to help us with our predictions.

In [6]:
# pulling important rows
transformed_country = Country_and_Subscriber_df[['External Video ID', 'Country Code', 'Views']]

# creating empty df
rangeRow = len(transformed_country['External Video ID'].unique())
test = pd.DataFrame(index = range(rangeRow), columns=['External Video ID','USA_views', 'IN_views','other_countries_views','total_views'], dtype = object)

# calc. values into empty df
testRow = 0 
for j in transformed_country['External Video ID'].unique():
    video_split = transformed_country[transformed_country['External Video ID'] == j]
    test.loc[testRow, 'External Video ID'] = j
    world_views = 0
    other_views = 0
    USA_views = 0
    IN_views = 0
    for i in range(len(video_split)):
        if video_split.iloc[i, 1] == 'US':
            USA_views = video_split.iloc[i, 2]
        elif video_split.iloc[i, 1] == 'IN':
            IN_views = video_split.iloc[i, 2]
        else:
            other_views = video_split.iloc[i, 2]
        world_views = world_views + other_views  
    test.loc[testRow,'USA_views'] = USA_views
    test.loc[testRow, 'IN_views'] = IN_views
    test.loc[testRow, 'other_countries_views'] = world_views
    test.loc[testRow, 'total_views'] = USA_views + IN_views + world_views
    testRow = testRow + 1

# finalizing data frame by finding percentages and removing unneeded columns
transformed_country = test
transformed_country['USA%_veiws'] = (transformed_country['USA_views'] / transformed_country['total_views']) * 100
transformed_country['IN%_views'] = (transformed_country['IN_views'] / transformed_country['total_views']) * 100
transformed_country['Other_country%_ views'] = (transformed_country['other_countries_views'] / transformed_country['total_views']) * 100
transformed_country.drop(['USA_views', 'IN_views', 'other_countries_views', 'total_views'], axis = 1, inplace = True)

In [7]:
transformed_country.head()

,External Video ID,USA%_veiws,IN%_views,Other_country%_ views
0,OtqQYqRNDGI,15.2449,23.9834,60.7718
1,_0rHU6qAQe0,16.761,12.2985,70.9405
2,4CpmB4TR2C4,6.87174,17.9558,75.1724
3,3ysXGYjvbSU,21.7565,5.78842,72.4551
4,s1gD35Z4eUc,11.1441,6.63136,82.2246


### Video_df

For this data frame, we need to remove the first row due to it giving column values and not row values. Then, we are going to change the video publish time to day of the week. We will also create a boolean feature which states if 'Data Science' appears in the title of the video or not. We also need to convert the average view duration to seconds rather than minute second format. Lastly, we will remove the total likes and dislikes column and replace it with a 'like_dislike_ratio' column. From this data frame, we want to attempt to predict the 'Your estimated revenue (USD)' feature. This feature tells us the estimated revenue of the video and will inform the video creator if they are being paid less or more than predicted by our model from historical data. To remove any features that will bias our results, we also want to remove 'RPM' and 'CPM' for giving away monetary information. 

In [8]:
# removing first row
transform_video = Video_df.iloc[1:,:]
transform_video.reset_index(inplace = True, drop = True)

# changing 'Video publish time' to day of the week
transform_video['Video pub­lish time'] = pd.to_datetime(transform_video['Video pub­lish time'])
transform_video['Video_Day_Published'] = transform_video['Video pub­lish time'].dt.day_name()

# we now want to create n-1 dummy features to replace 'Video_Day_Published'
dummy_day = pd.get_dummies(transform_video['Video_Day_Published'], drop_first = True)
transform_video = pd.concat([transform_video, dummy_day], axis = 1)

# next, we will created a boolean feature which specifies if 'Data Science' is in the title or not
test = range(len(transform_video['Video title']))
test = pd.DataFrame(test)
row = 0
for i in transform_video['Video title']:
    find_data = i.count('Data')
    find_science = i.count('Science')
    if find_data != 0 and find_science != 0:
        test.iloc[row, 0] = 1
    else:
        test.iloc[row, 0] = 0
    row = row + 1

test.rename(columns = {0: 'DataScience?'}, inplace = True)
transform_video = pd.concat([transform_video, test], axis = 1)

# changing average view duration to seconds format
test = range(len(transform_video['Av­er­age view dur­a­tion']))
test = pd.DataFrame(test)
row = 0
for i in transform_video['Av­er­age view dur­a­tion']:
    time = i.split(':')
    hour = int(time[0])
    hour_to_sec = hour * 3600
    min = int(time[1])
    min_to_sec = min * 60
    sec = int(time[2])
    total_time = hour_to_sec + min_to_sec + sec
    test.loc[row, 0] = total_time
    row = row + 1

test.rename(columns = {0: 'Average_view_duration_(s)'}, inplace = True)
transform_video = pd.concat([transform_video, test], axis = 1)

# finding the like to dislike ratio
transform_video['Like_Dislike_Ratio'] = round(transform_video['Likes'] / transform_video['Dis­likes'], 2)
row = 0
for i in transform_video['Like_Dislike_Ratio']:
    if i > 10000:
        transform_video.loc[row,'Like_Dislike_Ratio'] = transform_video.loc[row,'Likes'] 
    row = row + 1

# Now, we will remove features which are not needed
transform_video.drop(['Video title', 'Video pub­lish time', 'RPM (USD)', 'CPM (USD)', 'Video_Day_Published', 'Av­er­age view dur­a­tion', 'Likes', 'Dis­likes', 'Sub­scribers lost', 'Sub­scribers gained'], axis = 1, inplace = True)
transform_video.rename(columns = {'Video': 'External Video ID'}, inplace = True)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [9]:
transform_video.head()

,External Video ID,Com­ments ad­ded,Shares,Av­er­age per­cent­age viewed (%),Views,Watch time (hours),Sub­scribers,Your es­tim­ated rev­en­ue (USD),Im­pres­sions,Im­pres­sions click-through rate (%),Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,DataScience?,Average_view_duration_(s),Like_Dislike_Ratio
0,4OZip0cgOho,907,9583,36.65,1253559,65850.7042,46453,7959.533,26498799,3.14,0,0,0,0,0,0,1,189,49.79
1,78LjdAAw0wA,412,4,6.26,2291,200.2966,-3,6.113,188318,0.72,0,0,0,1,0,0,0,314,32.50
2,hO_YKK_0Qck,402,152,15.12,21350,3687.3387,189,202.963,442334,2.53,0,0,0,1,0,0,0,621,58.73
3,uXLnbdHMf8w,375,367,33.41,49564,2148.3110,1917,155.779,521185,4.01,0,1,0,0,0,0,1,156,119.18
4,Xgg7dIKys9E,329,118,9.55,13429,1034.3945,150,39.920,210876,3.38,0,0,0,0,0,1,0,277,39.33


### Performance_df

From this data frame, we can use the video length feature to help with our predictions. 

In [10]:
Performance_altered = Performance_df[['External Video ID', 'Video Length']]
transformed_performance = Performance_altered.groupby(by = 'External Video ID').max()

In [11]:
transformed_performance.head()

,Video Length
External Video ID,
-3d1NctSv0c,3413
-ONQ628CXKQ,964
-kX2b6TF_9k,656
-pdXWmj9xxU,866
-zbLpoJVBMI,368


This is the only new information we can retrieve from this data set due to having to aggregate based off video ID.

## Merging Data into one Data Frame, Normalizing, Splitting Data, and Writing to S3 Bucket

In [20]:
# first, we will merge the 4 data sets into one
df_merge = pd.merge(transformed_performance, transformed_country, on = 'External Video ID')
df_merge2 = pd.merge(transform_video, transformed_comments, on = 'External Video ID')
df_merge_final = pd.merge(df_merge, df_merge2, on = 'External Video ID')

# dropping video id
df_merge_final.drop(['External Video ID'], axis  = 1, inplace = True)

# The last step is to normalize our data due to having different units
normalized_df = df_merge_final[['Video Length', 'USA%_veiws', 'IN%_views', 'Other_country%_ views', 'Com­ments ad­ded', 'Shares', 'Av­er­age per­cent­age viewed (%)', 
                                'Views', 'Watch time (hours)', 'Sub­scribers', 'Im­pres­sions', 'Im­pres­sions click-through rate (%)', 'Average_view_duration_(s)', 
                                'Like_Dislike_Ratio', 'Reply_Comment_Count', 'Like_Comment_Count']]
not_normalized_df = df_merge_final[['Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday', 'DataScience?', 'Your es­tim­ated rev­en­ue (USD)']]
normalized_df = (normalized_df-normalized_df.mean())/normalized_df.std()
final_df = pd.concat([normalized_df, not_normalized_df], axis = 1)

# Splitting data
training, rem = train_test_split(final_df, train_size=0.8, random_state=42)
valid, test = train_test_split(rem, train_size=0.5, random_state=42)
training = pd.concat([training, training, training], axis = 0)

# uploading training to S3 bucket
csv_buffer = StringIO()
training.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(BUCKET, 'training_df.csv').put(Body = csv_buffer.getvalue())

# uploading validation to S3 bucket
csv_buffer = StringIO()
valid.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(BUCKET, 'validation_df.csv').put(Body = csv_buffer.getvalue())

# uploading testing to S3 bucket
csv_buffer = StringIO()
test.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(BUCKET, 'testing_df.csv').put(Body = csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'HC5GT893PGXG0A4X',
  'HostId': 'lCEwK7qlEU9lS5HhvVuBbHYfm8LGAa7kZ8qjrU9nHabKFgMKYt2rSI6WbfhGO91WSZZ0eDaQYOs=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'lCEwK7qlEU9lS5HhvVuBbHYfm8LGAa7kZ8qjrU9nHabKFgMKYt2rSI6WbfhGO91WSZZ0eDaQYOs=',
   'x-amz-request-id': 'HC5GT893PGXG0A4X',
   'date': 'Mon, 28 Mar 2022 21:23:03 GMT',
   'etag': '"f6b7b796698b2215c03e1ed712476874"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"f6b7b796698b2215c03e1ed712476874"'}